# Capturng the Realistic Travel Time
_Using r5py library
## 0. Load the Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas.core.frame")
# Allow 12 GB of memory
import sys
sys.argv.append(["--max-memory", "12G"])
# R5Py for travel time matrix computation
from r5py import TransportNetwork, TravelTimeMatrixComputer,DetailedItinerariesComputer

# GeoPandas for spatial data manipulation
import geopandas as gpd
import pandas as pd

# General library
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import datetime 

## 1. Load the Dataset 
1. the centroid of origin hexagons
2. the centroid of destination hexagons (is split into 11 files)
3. OpenStreetNetwork data(pbf):greater-london-latest.osm.pbf
4. Public Transport Timetable(gtfs):GTFS_17072024.zip

In [17]:
# Create the origins and destinations GeoDataFrame for calculating the travel time matrix
origins = gpd.read_file('geojson_files/origin_point.geojson')
# Destinations:
destinations = gpd.read_file('geojson_files/destination_point_part_11.geojson')

In [18]:
# Check the first few rows of the origins datasets
origins['id'] = origins['Origin_Hex8']
origins['id'] = origins['id'].astype(str)
origins = origins.reset_index(drop =True)
origins

,Origin_Hex8,geometry,id
0,88194ac001fffff,POINT (530444.262 160052.357),88194ac001fffff
1,88194ac003fffff,POINT (530762.165 159196.473),88194ac003fffff
2,88194ac005fffff,POINT (529591.550 160262.141),88194ac005fffff
3,88194ac007fffff,POINT (529909.454 159406.225),88194ac007fffff
4,88194ac009fffff,POINT (530979.008 160698.415),88194ac009fffff
...,...,...,...
2489,88195db6b3fffff,POINT (536069.780 199029.086),88195db6b3fffff
2490,88195db6b5fffff,POINT (534901.441 200093.653),88195db6b5fffff
2491,88195db6b7fffff,POINT (535218.923 199240.045),88195db6b7fffff
2492,88195db6b9fffff,POINT (536285.611 200525.207),88195db6b9fffff


In [19]:
# Check the first few rows of the destinations datasets
destinations['id'] = destinations['Destination_Hex8']
destinations

,Origin_Hex8,Destination_Hex8,geometry,id
0,88194ac003fffff,88194ac02bfffff,POINT (529056.670 159615.995),88194ac02bfffff
1,88194ac003fffff,88194ac145fffff,POINT (524373.247 163879.377),88194ac145fffff
2,88194ac003fffff,88194ac151fffff,POINT (526715.088 161747.580),88194ac151fffff
3,88194ac003fffff,88194ac211fffff,POINT (535673.995 162652.457),88194ac211fffff
4,88194ac003fffff,88194ac219fffff,POINT (536208.211 163298.107),88194ac219fffff
...,...,...,...,...
104411,88195db6b9fffff,88195da40dfffff,POINT (524972.041 189369.701),88195da40dfffff
104412,88195db6b9fffff,88195da62dfffff,POINT (526474.662 193653.291),88195da62dfffff
104413,88195db6b9fffff,88195da693fffff,POINT (533057.711 184266.199),88195da693fffff
104414,88195db6b9fffff,88195db489fffff,POINT (533300.908 198165.539),88195db489fffff


In [20]:
# Plot the features for better understanding
origins.explore()

In [21]:
###### Create a TransportNetwork object in R5Py using the OSM data ######
# This provides a basis for analysis in the entire third section
transport_network = TransportNetwork(
    osm_pbf='Data/r5r/greater-london-latest.osm.pbf', # Path to the OSM PBF file
    gtfs=['Data/r5r/GTFS_17072024.zip'] # Path to the GTFS file
)

In [22]:
all_results = pd.DataFrame()

# Iterate over each origin hexagon
for origin_hex in origins['Origin_Hex8'].unique():
    # Filter out the corresponding origins and destinations
    origin_data = origins[origins['Origin_Hex8'] == origin_hex]
    destination_data = destinations[destinations['Origin_Hex8'] == origin_hex]

    # Create a TravelTimeMatrixComputer object
    travel_time_computer = TravelTimeMatrixComputer(
        transport_network=transport_network,
        origins=origin_data[['id', 'geometry']],
        destinations=destination_data[['id', 'geometry']],
        snap_to_network=True,
        departure=datetime.datetime(2023, 3, 16, 11, 00),
        transport_modes=['WALK','BICYCLE','TRANSIT'],
        speed_walking=4.8,
        speed_cycling=16,
        max_bicycle_traffic_stress = 2
    )

    # Calculate travel time
    result = travel_time_computer.compute_travel_times()

   # Append the results to all_results
    all_results = pd.concat([all_results, result], ignore_index=True)

# Save all results to a CSV file
all_results.to_csv("Data/r5r_travel_time/0801/all_travel_times_11.csv", index=False)
print("Saved all results to all_travel_times.csv")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/r5py/r5/regional_task.py:227: RuntimeWarning: Departure time 2023-03-16 11:00:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/r5py/r5/regional_task.py:227: RuntimeWarning: Departure time 2023-03-16 11:00:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/r5py/r5/regional_task.py:227: RuntimeWarning: Departure time 2023-03-16 11:00:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/r5py/r5/regional_task.py:227: RuntimeWarning: Departure time 2023-03-16 11:00:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(
/Lib

Saved all results to all_travel_times.csv


In [23]:
# check the results
all_15 =all_results[all_results['travel_time']< 16]
all_15

,from_id,to_id,travel_time
103,88194ac007fffff,88194ac039fffff,10.0
159,88194ac009fffff,88194ac01dfffff,8.0
161,88194ac009fffff,88194ac047fffff,10.0
217,88194ac00bfffff,88194ac011fffff,7.0
221,88194ac00bfffff,88194ac2a5fffff,13.0
...,...,...,...
103914,88195db583fffff,88195db585fffff,15.0
104013,88195db593fffff,88195db599fffff,14.0
104331,88195db6b3fffff,88195db4d3fffff,14.0
104333,88195db6b3fffff,88195db6b5fffff,15.0
